In [1]:
import oracledb
import pandas as pd

oracledb.init_oracle_client('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Oracle\\instantclient_21_13')

connection = oracledb.connect(user="RISKCUSTOM", password="xxRiscRccess174!", host="exatest2-scan.moscow.eurochem.ru", port=1521, service_name='riskdev.moscow.eurochem.ru', disable_oob= True)

In [81]:
query = """select * from "RISKACCESS"."XXMR_MADAB_CONTENT"
where
"RISKACCESS"."XXMR_MADAB_CONTENT"."COMMODITY_ID" in (2354,322,311,312,314,315,318,2332,2334,2360,9321,9326,9331,9902,10014,7647,33051,9318,9886,2362,33447) and
"RISKACCESS"."XXMR_MADAB_CONTENT"."PERIOD" = TO_DATE('29/02/24', 'DD/MM/YY')
"""

In [82]:
data_export = pd.read_sql(query, con=connection)
data_export.head(2)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_5172\3811057091.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,ID,INSERTED_AT,COMMODITY_ID,NAME,VALUE_QUALIFIER,PUBLICATION_DATE,PERIOD_FROM,PERIOD_TO,PERIOD,PROPOGATE_ID,ENTITY,VALUE1,VALUE2,SOURCE_ID,STATUS,PERMISSION,APPROVED_BY,APPROVED_AT
0,None,2024-03-21 08:57:23,311,last,last,2024-02-29,2024-02-29,2024-03-14,2024-02-29,1456616787,0,0.8845,None,367869,P,None,None,None
1,None,2024-03-14 19:22:40,312,last,last,2024-02-29,2024-02-29,2024-03-14,2024-02-29,1454532263,0,1.0805,None,363784,P,None,None,None


In [14]:
Data_currency = data_export[['COMMODITY_ID', 'VALUE1']]
Data_currency

,COMMODITY_ID,VALUE1
0,311,0.8845
1,312,1.0805
2,314,7.1886
3,315,1.2625
4,318,1.3579
5,322,91.2001
6,2332,4.9708
7,2334,82.9138
8,2354,1.5391
9,2360,149.9800


In [50]:
map_str = '''-- 322 = USD/RUB
-- 311 = USD/CHF
-- 312 = EUR/USD
-- 314 = USD/CNY
-- 315 = GBP/USD
-- 318 = USD/CAD
--2332 = USD/BRL
--2334 = USD/INR
--2354 = USD/AUD
--2360 = USD/JPY
--9321 = USD/KZT
--9326 = USD/PLN
--9331 = USD/TRY
--9902 = USD/KRW
--10014= USD/TWD
--33051= USD/AED
--9926 = EUR/CHF
--9318 = USD/BGN
--9886 = USD/MDL
--33447= USD/RON
--9911 = EUR/RSD
--9908 = EUR/HUF
--2362 = USD/SGD
--9329 = USD/ZAR'''

In [53]:
map_list = map_str.replace(' ','').split('\n')
map_dict = {}
for i in range(len(map_list)):
    mini_list = map_list[i].split(sep='--')[1].split('=')
    map_dict[mini_list[0]] = mini_list[1]

map_dict

{'322': 'USD/RUB',
 '311': 'USD/CHF',
 '312': 'EUR/USD',
 '314': 'USD/CNY',
 '315': 'GBP/USD',
 '318': 'USD/CAD',
 '2332': 'USD/BRL',
 '2334': 'USD/INR',
 '2354': 'USD/AUD',
 '2360': 'USD/JPY',
 '9321': 'USD/KZT',
 '9326': 'USD/PLN',
 '9331': 'USD/TRY',
 '9902': 'USD/KRW',
 '10014': 'USD/TWD',
 '33051': 'USD/AED',
 '9926': 'EUR/CHF',
 '9318': 'USD/BGN',
 '9886': 'USD/MDL',
 '33447': 'USD/RON',
 '9911': 'EUR/RSD',
 '9908': 'EUR/HUF',
 '2362': 'USD/SGD',
 '9329': 'USD/ZAR'}

In [74]:
CCY_df = pd.DataFrame({'id':map_dict.keys(), 'CCY': map_dict.values()}, index=[x for x in range(len(map_dict.keys()))])
CCY_df.id = CCY_df.id.astype(int)
merge_df = Data_currency.merge(CCY_df, how='left', left_on='COMMODITY_ID', right_on='id').drop(columns='id')
merge_df

,COMMODITY_ID,VALUE1,CCY
0,311,0.8845,USD/CHF
1,312,1.0805,EUR/USD
2,314,7.1886,USD/CNY
3,315,1.2625,GBP/USD
4,318,1.3579,USD/CAD
5,322,91.2001,USD/RUB
6,2332,4.9708,USD/BRL
7,2334,82.9138,USD/INR
8,2354,1.5391,USD/AUD
9,2360,149.9800,USD/JPY


In [86]:
CCY_df.to_excel('CCY_mapping.xlsx', index=False)

In [85]:
query_2 = '''
select *
from "RISKACCESS"."XXMR_MADAB_COMMODITIES"'''
data_export_2 = pd.read_sql(query_2, con=connection)
data_export_2.head(2)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_5172\2698327378.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export_2 = pd.read_sql(query_2, con=connection)


,ID,CODE,DESCRIPTION,PRODUCT_CATEGORY,PRODUCT_GROUP,PRODUCT_BRIEF_NAME,PRODUCT_NAME,REGION,COMMODITY_CODE_LOCATION,GEOPOINT,...,UNIT,CURRENCY_1,CURRENCY_2,MULTIPLIER,CALENDAR_1,CALENDAR_2,FORWARD_BY,VALID_FROM,VALID_TO,UUID
0,109,AS China,Market AS China FOB A Spot Weekly USD/mt,Fertilizer,Nitrogen Fertilizers,Ammonium Sulphate,AS,Asia&Pacific,China,n/a,...,mt,USD,None,NaN,None,None,S,1900-01-01,4000-01-01 00:00:00,AS China/Market/ICIS/FOB/A Spot Weekly/USD/mt/S
1,110,AS Europe,Market AS Europe FOB A Spot Weekly USD/mt,Fertilizer,Nitrogen Fertilizers,Ammonium Sulphate,AS,Europe,Europe,n/a,...,mt,USD,None,NaN,None,None,S,1900-01-01,4000-01-01 00:00:00,AS Europe/Market/ICIS/FOB/A Spot Weekly/USD/mt/S
